In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.cross_validation import train_test_split

import time 
start_time = time.time()

/Users/iTenki/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv("/Users/iTenki/Downloads/train.csv")
tests = pd.read_csv("/Users/iTenki/Downloads/test.csv")

In [3]:
params={
'booster':'gbtree',
'objective': 'multi:softmax', #多分类的问题
'num_class':10, # 类别数，与 multisoftmax 并用
'gamma':0.1,  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
'max_depth':12, # 构建树的深度，越大越容易过拟合
'lambda':2,  # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
'subsample':0.7, # 随机采样训练样本
'colsample_bytree':0.7, # 生成树时进行的列采样
'min_child_weight':3, 
# 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言
#，假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。
#这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。 
'silent':0 ,#设置成1则没有运行信息输出，最好是设置为0.
'eta': 0.007, # 如同学习率
'seed':1000,
'nthread':7,# cpu 线程数
#'eval_metric': 'auc'
}

In [4]:
plst = list(params.items())
num_rounds = 5000 # 迭代次数

In [5]:
train_xy,val = train_test_split(train, test_size = 0.3,random_state=1)
#random_state is of big influence for val-auc
y = train_xy.label
X = train_xy.drop(['label'],axis=1)
val_y = val.label
val_X = val.drop(['label'],axis=1)

In [6]:
xgb_val = xgb.DMatrix(val_X,label=val_y)
xgb_train = xgb.DMatrix(X, label=y)
xgb_test = xgb.DMatrix(tests)

In [7]:
watchlist = [(xgb_train, 'train'),(xgb_val, 'val')]

In [8]:
# training model 
# early_stopping_rounds 当设置的迭代次数较大时，early_stopping_rounds 可在一定的迭代次数内准确率没有提升就停止训练
model = xgb.train(plst, xgb_train, num_rounds, watchlist,early_stopping_rounds=100)

[0]	train-merror:0.086293	val-merror:0.125238
Multiple eval metrics have been passed: 'val-merror' will be used for early stopping.

Will train until val-merror hasn't improved in 100 rounds.
[1]	train-merror:0.062891	val-merror:0.098333
[2]	train-merror:0.054898	val-merror:0.087302
[3]	train-merror:0.050238	val-merror:0.080397
[4]	train-merror:0.047245	val-merror:0.078333
[5]	train-merror:0.046497	val-merror:0.07619
[6]	train-merror:0.044014	val-merror:0.074444
[7]	train-merror:0.043673	val-merror:0.072143
[8]	train-merror:0.042959	val-merror:0.072063
[9]	train-merror:0.042279	val-merror:0.071508
[10]	train-merror:0.041395	val-merror:0.070159
[11]	train-merror:0.040952	val-merror:0.070159
[12]	train-merror:0.040442	val-merror:0.070794
[13]	train-merror:0.039864	val-merror:0.068968
[14]	train-merror:0.039728	val-merror:0.068413
[15]	train-merror:0.039558	val-merror:0.067857
[16]	train-merror:0.038912	val-merror:0.067619
[17]	train-merror:0.038776	val-merror:0.06746
[18]	train-merror:0.

[171]	train-merror:0.019354	val-merror:0.050556
[172]	train-merror:0.019252	val-merror:0.050476
[173]	train-merror:0.019218	val-merror:0.050556
[174]	train-merror:0.019082	val-merror:0.050635
[175]	train-merror:0.019014	val-merror:0.050397
[176]	train-merror:0.018946	val-merror:0.050476
[177]	train-merror:0.018844	val-merror:0.050317
[178]	train-merror:0.01881	val-merror:0.050317
[179]	train-merror:0.018878	val-merror:0.050635
[180]	train-merror:0.018571	val-merror:0.050476
[181]	train-merror:0.018537	val-merror:0.050238
[182]	train-merror:0.018469	val-merror:0.050238
[183]	train-merror:0.018503	val-merror:0.050397
[184]	train-merror:0.018401	val-merror:0.050159
[185]	train-merror:0.018435	val-merror:0.050317
[186]	train-merror:0.018333	val-merror:0.050238
[187]	train-merror:0.018299	val-merror:0.050159
[188]	train-merror:0.018197	val-merror:0.050079
[189]	train-merror:0.018197	val-merror:0.050079
[190]	train-merror:0.018163	val-merror:0.049921
[191]	train-merror:0.018163	val-merror:0.

[343]	train-merror:0.011463	val-merror:0.04381
[344]	train-merror:0.011463	val-merror:0.04373
[345]	train-merror:0.011361	val-merror:0.043651
[346]	train-merror:0.011361	val-merror:0.043651
[347]	train-merror:0.011293	val-merror:0.043571
[348]	train-merror:0.011259	val-merror:0.043492
[349]	train-merror:0.011293	val-merror:0.043413
[350]	train-merror:0.011224	val-merror:0.043413
[351]	train-merror:0.01119	val-merror:0.043571
[352]	train-merror:0.011156	val-merror:0.043413
[353]	train-merror:0.011122	val-merror:0.043333
[354]	train-merror:0.011156	val-merror:0.043333
[355]	train-merror:0.011156	val-merror:0.043333
[356]	train-merror:0.011122	val-merror:0.043254
[357]	train-merror:0.01102	val-merror:0.043254
[358]	train-merror:0.010918	val-merror:0.043333
[359]	train-merror:0.010816	val-merror:0.043333
[360]	train-merror:0.010816	val-merror:0.043254
[361]	train-merror:0.010816	val-merror:0.043254
[362]	train-merror:0.010782	val-merror:0.043175
[363]	train-merror:0.01068	val-merror:0.0431

[515]	train-merror:0.006054	val-merror:0.040238
[516]	train-merror:0.006054	val-merror:0.040317
[517]	train-merror:0.006054	val-merror:0.040317
[518]	train-merror:0.00602	val-merror:0.040238
[519]	train-merror:0.006054	val-merror:0.04
[520]	train-merror:0.006054	val-merror:0.039921
[521]	train-merror:0.006054	val-merror:0.040159
[522]	train-merror:0.005952	val-merror:0.039921
[523]	train-merror:0.005884	val-merror:0.039921
[524]	train-merror:0.00585	val-merror:0.039762
[525]	train-merror:0.00585	val-merror:0.039762
[526]	train-merror:0.005816	val-merror:0.039841
[527]	train-merror:0.005816	val-merror:0.039603
[528]	train-merror:0.005816	val-merror:0.039603
[529]	train-merror:0.005816	val-merror:0.039524
[530]	train-merror:0.005816	val-merror:0.039365
[531]	train-merror:0.005816	val-merror:0.039365
[532]	train-merror:0.005748	val-merror:0.039365
[533]	train-merror:0.005748	val-merror:0.039365
[534]	train-merror:0.005714	val-merror:0.039365
[535]	train-merror:0.005714	val-merror:0.039286

[687]	train-merror:0.003333	val-merror:0.036429
[688]	train-merror:0.003299	val-merror:0.036508
[689]	train-merror:0.003299	val-merror:0.036349
[690]	train-merror:0.003265	val-merror:0.036429
[691]	train-merror:0.003265	val-merror:0.036349
[692]	train-merror:0.003265	val-merror:0.036349
[693]	train-merror:0.003265	val-merror:0.03627
[694]	train-merror:0.003265	val-merror:0.03627
[695]	train-merror:0.003265	val-merror:0.03627
[696]	train-merror:0.003231	val-merror:0.03627
[697]	train-merror:0.003231	val-merror:0.036349
[698]	train-merror:0.003231	val-merror:0.036349
[699]	train-merror:0.003231	val-merror:0.036349
[700]	train-merror:0.003231	val-merror:0.036349
[701]	train-merror:0.003231	val-merror:0.03627
[702]	train-merror:0.003163	val-merror:0.036111
[703]	train-merror:0.003163	val-merror:0.03619
[704]	train-merror:0.003163	val-merror:0.036111
[705]	train-merror:0.003129	val-merror:0.036111
[706]	train-merror:0.003129	val-merror:0.036032
[707]	train-merror:0.003095	val-merror:0.03611

...

[1368]	train-merror:0.000238	val-merror:0.029762
[1369]	train-merror:0.000238	val-merror:0.029762
[1370]	train-merror:0.000238	val-merror:0.029841
[1371]	train-merror:0.000238	val-merror:0.029762
[1372]	train-merror:0.000238	val-merror:0.029762
[1373]	train-merror:0.000238	val-merror:0.029762
[1374]	train-merror:0.000238	val-merror:0.029762
[1375]	train-merror:0.000238	val-merror:0.029762
[1376]	train-merror:0.000238	val-merror:0.029762
[1377]	train-merror:0.000238	val-merror:0.029762
[1378]	train-merror:0.000238	val-merror:0.029762
[1379]	train-merror:0.000238	val-merror:0.029762
[1380]	train-merror:0.000238	val-merror:0.029762
[1381]	train-merror:0.000238	val-merror:0.029762
[1382]	train-merror:0.000238	val-merror:0.029762
[1383]	train-merror:0.000238	val-merror:0.029762
[1384]	train-merror:0.000204	val-merror:0.029762
[1385]	train-merror:0.000204	val-merror:0.029762
[1386]	train-merror:0.000204	val-merror:0.029762
[1387]	train-merror:0.000204	val-merror:0.029762
[1388]	train-merror:

[1536]	train-merror:0.000136	val-merror:0.029286
[1537]	train-merror:0.000136	val-merror:0.029286
[1538]	train-merror:0.000136	val-merror:0.029286
[1539]	train-merror:0.000136	val-merror:0.029286
[1540]	train-merror:0.000136	val-merror:0.029286
[1541]	train-merror:0.000136	val-merror:0.029286
[1542]	train-merror:0.000136	val-merror:0.029286
[1543]	train-merror:0.000136	val-merror:0.029286
[1544]	train-merror:0.000136	val-merror:0.029286
[1545]	train-merror:0.000136	val-merror:0.029286
[1546]	train-merror:0.000136	val-merror:0.029286
[1547]	train-merror:0.000136	val-merror:0.029286
[1548]	train-merror:0.000136	val-merror:0.029286
[1549]	train-merror:0.000136	val-merror:0.029286
[1550]	train-merror:0.000136	val-merror:0.029286
[1551]	train-merror:0.000136	val-merror:0.029286
[1552]	train-merror:0.000136	val-merror:0.029286
[1553]	train-merror:0.000136	val-merror:0.029286
[1554]	train-merror:0.000102	val-merror:0.029286
[1555]	train-merror:0.000102	val-merror:0.029286
[1556]	train-merror:

[1706]	train-merror:3.4e-05	val-merror:0.028492
[1707]	train-merror:3.4e-05	val-merror:0.028492
[1708]	train-merror:3.4e-05	val-merror:0.028651
[1709]	train-merror:3.4e-05	val-merror:0.028571
[1710]	train-merror:3.4e-05	val-merror:0.02873
[1711]	train-merror:3.4e-05	val-merror:0.028571
[1712]	train-merror:3.4e-05	val-merror:0.028571
[1713]	train-merror:3.4e-05	val-merror:0.028571
[1714]	train-merror:3.4e-05	val-merror:0.028571
[1715]	train-merror:3.4e-05	val-merror:0.028492
[1716]	train-merror:3.4e-05	val-merror:0.028571
[1717]	train-merror:3.4e-05	val-merror:0.028571
[1718]	train-merror:3.4e-05	val-merror:0.028571
[1719]	train-merror:3.4e-05	val-merror:0.028571
[1720]	train-merror:3.4e-05	val-merror:0.028571
[1721]	train-merror:3.4e-05	val-merror:0.028571
[1722]	train-merror:3.4e-05	val-merror:0.028571
[1723]	train-merror:3.4e-05	val-merror:0.028651
[1724]	train-merror:3.4e-05	val-merror:0.028651
[1725]	train-merror:3.4e-05	val-merror:0.02873
[1726]	train-merror:3.4e-05	val-merror:0.0

[1878]	train-merror:3.4e-05	val-merror:0.028254
[1879]	train-merror:3.4e-05	val-merror:0.028175
[1880]	train-merror:3.4e-05	val-merror:0.028175
[1881]	train-merror:3.4e-05	val-merror:0.028175
[1882]	train-merror:3.4e-05	val-merror:0.028175
[1883]	train-merror:3.4e-05	val-merror:0.028175
[1884]	train-merror:3.4e-05	val-merror:0.028254
[1885]	train-merror:3.4e-05	val-merror:0.028175
[1886]	train-merror:3.4e-05	val-merror:0.028175
[1887]	train-merror:3.4e-05	val-merror:0.028175
[1888]	train-merror:3.4e-05	val-merror:0.028175
[1889]	train-merror:3.4e-05	val-merror:0.028095
[1890]	train-merror:3.4e-05	val-merror:0.028095
[1891]	train-merror:3.4e-05	val-merror:0.028095
[1892]	train-merror:3.4e-05	val-merror:0.028095
[1893]	train-merror:3.4e-05	val-merror:0.028175
[1894]	train-merror:3.4e-05	val-merror:0.028175
[1895]	train-merror:3.4e-05	val-merror:0.028095
[1896]	train-merror:3.4e-05	val-merror:0.028095
[1897]	train-merror:3.4e-05	val-merror:0.028095
[1898]	train-merror:3.4e-05	val-merror:0

[2049]	train-merror:0	val-merror:0.028016
[2050]	train-merror:0	val-merror:0.028016
[2051]	train-merror:0	val-merror:0.027937
[2052]	train-merror:0	val-merror:0.027937
[2053]	train-merror:0	val-merror:0.027937
[2054]	train-merror:0	val-merror:0.027937
[2055]	train-merror:0	val-merror:0.027937
[2056]	train-merror:0	val-merror:0.027937
[2057]	train-merror:0	val-merror:0.027937
[2058]	train-merror:0	val-merror:0.027937
[2059]	train-merror:0	val-merror:0.027857
[2060]	train-merror:0	val-merror:0.027857
[2061]	train-merror:0	val-merror:0.027857
[2062]	train-merror:0	val-merror:0.027857
[2063]	train-merror:0	val-merror:0.027857
[2064]	train-merror:0	val-merror:0.027857
[2065]	train-merror:0	val-merror:0.027857
[2066]	train-merror:0	val-merror:0.027857
[2067]	train-merror:0	val-merror:0.027857
[2068]	train-merror:0	val-merror:0.027857
[2069]	train-merror:0	val-merror:0.027857
[2070]	train-merror:0	val-merror:0.027857
[2071]	train-merror:0	val-merror:0.027857
[2072]	train-merror:0	val-merror:0

In [9]:
preds = model.predict(xgb_test,ntree_limit=model.best_ntree_limit)

In [10]:
np.savetxt('xgb_submission.csv',np.c_[range(1,len(tests)+1),preds],delimiter=',',header='ImageId,Label',comments='',fmt='%d')

In [11]:
#输出运行时长
cost_time = time.time()-start_time
print "xgboost success!",'\n',"cost time:",cost_time,"(s)"

SyntaxError: invalid syntax (<ipython-input-11-dedf0d511b14>, line 3)